In [6]:
from hgcal_dev.datasets.vertex import VertexDataModule
import numpy as np
import pandas as pd
import torch
import plotly
import plotly.express as px
from pathlib import Path
from hdbscan import HDBSCAN
from hgcal_dev.metrics.instance import match_instances
from tqdm import tqdm
from sklearn.manifold import TSNE
from omegaconf import OmegaConf
import hydra
from hydra.experimental import initialize, compose
from sklearn.cluster import MeanShift
from hgcal_dev.metrics.instance import PanopticQuality
from tqdm import tqdm

In [7]:
def clean_events():
    clean_path = Path('/home/alexj/data/hgcal-dev/vertex/clean')
    data_path = Path('/home/alexj/data/hgcal-dev/vertex/data')
    for path in tqdm(data_path.glob('*.pkl')):
        df = pd.read_pickle(path)
        clusters = df['vertex_id'].unique()
        output_path = clean_path / path.name
        good_clusters = []
        for c in clusters:
            if df.loc[(df['vertex_id'] == c), 'Z'].unique().shape[0] == 1:
                good_clusters.append(c)
        clean_df = df[np.isin(df['vertex_id'], good_clusters)]
        clean_df.reset_index(inpalce=True, drop=True)
        if clean_df.shape[0] > 0:
            clean_df.to_pickle(output_path)


In [30]:
clean_events()

1078it [01:28, 13.00it/s]

In [1]:
data_path = Path('/home/alexj/data/hgcal-dev/vertex/clean')
dfs = [pd.read_pickle(f) for f in [e for e in data_path.glob('*.pkl')][:10]]
#df = pd.concat(dfs, ignore_index=True)

NameError: name 'Path' is not defined

In [ ]:
def frac(df):
    clusters = df['vertex_id'].unique()
    return sum((df.loc[(df['vertex_id'] == c), 'Z'].unique().shape[0] == 1) for c in clusters) / clusters.shape[0]

In [ ]:
sum(frac(df) for df in dfs) / len(dfs)

In [ ]:
clusterer = MeanShift(bandwidth=0.004)

In [ ]:
pred = clusterer.fit_predict(dfs[0]['vertex_id'].values.reshape(-1, 1))

In [ ]:
plot_df = pd.DataFrame({'true': dfs[0]['vertex_id'], 'pred': pred, 'Z': dfs[0]['Z']})

In [ ]:
pqs = []
for df in dfs:
    pred = clusterer.fit_predict(df['vertex_id'].values.reshape(-1, 1))
    metric = PanopticQuality(num_classes=1, semantic=False)
    metric.add(outputs=pred, targets=df['vertex_id'].values)
    metrics = metric.compute()
    pqs.append(metrics['pq'])

In [ ]:
pqs

In [21]:
df['Z'].min()

-1064532.9

In [26]:
(df['Z'] > 1e6).sum() / df.shape[0] * 100

1.1529221532550595e-05